### Import packages

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
import plotly.express as px

import copy
import seaborn as sns
import os
from scipy import stats
import datetime

import sqlite3 as sqlite
from sqlalchemy import create_engine
%matplotlib inline

In [24]:
data_path = '/mnt/d/lighthouse/Midterm_data/'

In [25]:
# 15927485 rows × 42
flights = pd.read_csv(data_path + 'X_cleaned (1).csv', header = 0)
flights.drop('Unnamed: 0', axis = 1, inplace = True)
flights.head()

,distance,latitude,longitude,wdir,temp,visibility,wspd,heatindex,cloudcover,precip,...,origin,dest,branded_code_share,mkt_carrier_fl_num,snow_type,month,day,mkt_carrier_fl_num.1,arr_hour,dep_hour
0,-0.696674,-0.453104,-1.18607,-0.895622,1.110527,-0.618003,-0.403782,-1.593617,0.030259,0.985565,...,0,0,0,5888.0,0.0,1,1.0,5888.0,19,18
1,-0.696674,-0.453104,-1.18607,-0.895622,1.110527,-0.618003,-0.403782,-1.593617,0.030259,0.985565,...,0,0,0,6276.0,0.0,1,1.0,6276.0,13,11
2,-0.748596,-0.453104,-1.18607,-0.895622,1.110527,-0.618003,-0.403782,-1.593617,0.030259,0.985565,...,0,1,0,4598.0,0.0,1,1.0,4598.0,21,20
3,-0.748596,-0.453104,-1.18607,-0.895622,1.110527,-0.618003,-0.403782,-1.593617,0.030259,0.985565,...,0,1,0,4761.0,0.0,1,1.0,4761.0,14,13
4,-0.748596,-0.453104,-1.18607,-0.895622,1.110527,-0.618003,-0.403782,-1.593617,0.030259,0.985565,...,0,1,0,5162.0,0.0,1,1.0,5162.0,10,9


In [27]:
flights.columns

Index(['distance', 'latitude', 'longitude', 'wdir', 'temp', 'visibility',
       'wspd', 'heatindex', 'cloudcover', 'precip', 'sealevelpressure',
       'humidity', 'wgust', 'precipcover', 'windchill', 'total_flights',
       'mean_tail_num_arr_delay', 'mean_carrier_arr_delay', 'origin', 'dest',
       'branded_code_share', 'mkt_carrier_fl_num', 'snow_type', 'month', 'day',
       'mkt_carrier_fl_num.1', 'arr_hour', 'dep_hour'],
      dtype='object')

In [37]:
# 29
i = ['distance', 'latitude', 'longitude', 'wdir', 'temp', 'visibility',
       'wspd', 'heatindex', 'cloudcover', 'precip', 'sealevelpressure',
       'humidity', 'wgust', 'precipcover', 'windchill', 'total_flights',
       'mean_tail_num_arr_delay', 'mean_carrier_arr_delay', 'origin', 'dest',
       'branded_code_share', 'mkt_carrier_fl_num', 'snow_type', 'month', 'day',
       'arr_hour', 'dep_hour']

In [35]:
# 28
j = ['mkt_carrier_fl_num', 'crs_dep_time', 'crs_arr_time', 'distance',
       'year', 'month', 'date', 'latitude', 'longitude', 'wdir', 'temp',
       'visibility', 'wspd', 'heatindex', 'cloudcover', 'precip',
       'sealevelpressure', 'humidity', 'wgust', 'precipcover', 'windchill',
       'total_flights', 'snow_type', 'branded_code_share', 'origin', 'dest',
       'mean_tail_num_arr_delay', 'mean_carrier_arr_delay']

In [38]:
set(i)^set(j)

{'arr_hour', 'crs_arr_time', 'crs_dep_time', 'date', 'day', 'dep_hour', 'year'}

In [4]:
# making duplicate so we dont have to re-read large csv files
flights_clean = copy.deepcopy(flights)

## Nan

In [5]:
total = flights_clean.isnull().sum().sort_values(ascending = False)
percent = (flights_clean.isnull().sum()/flights_clean.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])

In [6]:
#missing_data.head(15)

In [6]:
# dropping columsn with 80% missing AND [origin_city_name, dest_city_name] since we have [orgin, dest]
# dropping [cancelled, diverted, flights, dup] since they only contain one value
cols_to_drop_nan = list(missing_data[missing_data['Percent'] > 0.8].index)
cols_to_drop_other = ['dep_delay','taxi_out','taxi_in', 'wheels_off', 'wheels_on', 'arr_time', 'cancelled', 'diverted','actual_elapsed_time','air_time', 'flights', 'dup', 'dep_time']
flights_clean.drop(cols_to_drop_nan, axis = 1, inplace = True)
flights_clean.drop(cols_to_drop_other, axis = 1, inplace = True)
flights_test.drop(['dup','flights'], axis =1, inplace=True)

## rows with Nan

In [33]:
# before: (15927485, 20)
# after: 15652397 rows × 20 columns
# lost 3% of data, no biggie
flights_clean.dropna(inplace = True)

In [34]:
# (15605076, 19)
flights_clean

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance
0,2019-03-25,WN,WN,WN,2098,WN,N8540V,2098,13198,MCI,"Kansas City, MO",14107,PHX,"Phoenix, AZ",1755,1850,-12.0,175.0,1044
1,2019-03-25,WN,WN,WN,2238,WN,N8656B,2238,13198,MCI,"Kansas City, MO",14107,PHX,"Phoenix, AZ",2000,2055,-19.0,175.0,1044
2,2019-03-25,WN,WN,WN,2451,WN,N8583Z,2451,13198,MCI,"Kansas City, MO",14107,PHX,"Phoenix, AZ",540,635,-10.0,175.0,1044
3,2019-03-25,WN,WN,WN,2213,WN,N737JW,2213,13198,MCI,"Kansas City, MO",14492,RDU,"Raleigh/Durham, NC",1550,1905,94.0,135.0,904
4,2019-03-25,WN,WN,WN,2096,WN,N705SW,2096,13198,MCI,"Kansas City, MO",14635,RSW,"Fort Myers, FL",1045,1430,152.0,165.0,1155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15927480,2019-03-25,WN,WN,WN,2189,WN,N7702A,2189,13198,MCI,"Kansas City, MO",13495,MSY,"New Orleans, LA",2055,2245,-3.0,110.0,689
15927481,2019-03-25,WN,WN,WN,1291,WN,N7878A,1291,13198,MCI,"Kansas City, MO",13796,OAK,"Oakland, CA",1200,1355,99.0,235.0,1489
15927482,2019-03-25,WN,WN,WN,2470,WN,N760SW,2470,13198,MCI,"Kansas City, MO",14057,PDX,"Portland, OR",920,1110,-18.0,230.0,1482
15927483,2019-03-25,WN,WN,WN,1651,WN,N8542Z,1651,13198,MCI,"Kansas City, MO",14107,PHX,"Phoenix, AZ",1125,1215,-11.0,170.0,1044


### seperate numerical/categorical

In [35]:
# 9 numerical features
numerical = list(flights_clean.dtypes[flights_clean.dtypes != 'object'].index)
# 10 categorical features (10 if include city and state)
categorical = list(flights_clean.dtypes[flights_clean.dtypes == 'object'].index)

In [36]:
flights_numerical = copy.deepcopy(flights_clean[numerical])
flights_categorical = copy.deepcopy(flights_clean[categorical])

### seperate year-month-day

In [37]:
flights_categorical['fl_date'] = pd.to_datetime(flights_categorical['fl_date'])
flights_categorical['year'] = flights_categorical['fl_date'].dt.year
flights_categorical['month'] = flights_categorical['fl_date'].dt.month
flights_categorical['date'] = flights_categorical['fl_date'].dt.day
# Dropping the old date column
flights_categorical.drop('fl_date', axis = 1, inplace = True)

In [38]:
flights_categorical

,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,year,month,date
0,WN,WN,WN,WN,N8540V,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25
1,WN,WN,WN,WN,N8656B,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25
2,WN,WN,WN,WN,N8583Z,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25
3,WN,WN,WN,WN,N737JW,MCI,"Kansas City, MO",RDU,"Raleigh/Durham, NC",2019,3,25
4,WN,WN,WN,WN,N705SW,MCI,"Kansas City, MO",RSW,"Fort Myers, FL",2019,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...
15927480,WN,WN,WN,WN,N7702A,MCI,"Kansas City, MO",MSY,"New Orleans, LA",2019,3,25
15927481,WN,WN,WN,WN,N7878A,MCI,"Kansas City, MO",OAK,"Oakland, CA",2019,3,25
15927482,WN,WN,WN,WN,N760SW,MCI,"Kansas City, MO",PDX,"Portland, OR",2019,3,25
15927483,WN,WN,WN,WN,N8542Z,MCI,"Kansas City, MO",PHX,"Phoenix, AZ",2019,3,25


### encode all string variables

In [39]:
# converting category to numbers
flights_categorical['mkt_unique_carrier'] = pd.factorize(flights_categorical['mkt_unique_carrier'])[0]
flights_categorical['branded_code_share'] = pd.factorize(flights_categorical['branded_code_share'])[0]
flights_categorical['mkt_carrier'] = pd.factorize(flights_categorical['mkt_carrier'])[0]
flights_categorical['op_unique_carrier'] = pd.factorize(flights_categorical['op_unique_carrier'])[0]
flights_categorical['tail_num'] = pd.factorize(flights_categorical['tail_num'])[0]
flights_categorical['origin'] = pd.factorize(flights_categorical['origin'])[0]
flights_categorical['origin_city_name'] = pd.factorize(flights_categorical['origin_city_name'])[0]
flights_categorical['dest'] = pd.factorize(flights_categorical['dest'])[0]
flights_categorical['dest_city_name'] = pd.factorize(flights_categorical['dest_city_name'])[0]

### merging into into model data

In [40]:
flights_final = pd.concat([flights_numerical, flights_categorical], axis=1)
flights_final.reset_index(drop=True)

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,mkt_unique_carrier,...,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,year,month,date
0,2098,2098,13198,14107,1755,1850,-12.0,175.0,1044,0,...,0,0,0,0,0,0,0,2019,3,25
1,2238,2238,13198,14107,2000,2055,-19.0,175.0,1044,0,...,0,0,1,0,0,0,0,2019,3,25
2,2451,2451,13198,14107,540,635,-10.0,175.0,1044,0,...,0,0,2,0,0,0,0,2019,3,25
3,2213,2213,13198,14492,1550,1905,94.0,135.0,904,0,...,0,0,3,0,0,1,1,2019,3,25
4,2096,2096,13198,14635,1045,1430,152.0,165.0,1155,0,...,0,0,4,0,0,2,2,2019,3,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15615735,2189,2189,13198,13495,2055,2245,-3.0,110.0,689,0,...,0,0,143,0,0,30,30,2019,3,25
15615736,1291,1291,13198,13796,1200,1355,99.0,235.0,1489,0,...,0,0,197,0,0,31,31,2019,3,25
15615737,2470,2470,13198,14057,920,1110,-18.0,230.0,1482,0,...,0,0,290,0,0,58,58,2019,3,25
15615738,1651,1651,13198,14107,1125,1215,-11.0,170.0,1044,0,...,0,0,293,0,0,0,0,2019,3,25


In [41]:
#flights_final.to_csv('model_1_encode_only.csv')

# Model

In [7]:
flights_final = pd.read_csv(data_path + 'model_1_encode_only.csv', header = 0)
flights_final.drop('Unnamed: 0', axis = 1, inplace= True)

In [8]:
flights_test =pd.read_csv(data_path + 'flights_test.csv', header = 0)
flights_test.drop(['dup','flights'], axis =1, inplace=True)
flights_test['fl_date'] = pd.to_datetime(flights_test['fl_date'])
flights_test['year'] = flights_test['fl_date'].dt.year
flights_test['month'] = flights_test['fl_date'].dt.month
flights_test['date'] = flights_test['fl_date'].dt.day
# Dropping the old date column
flights_test.drop('fl_date', axis = 1, inplace = True)

In [9]:
flights_final.head()

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,crs_arr_time,arr_delay,crs_elapsed_time,distance,mkt_unique_carrier,...,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,year,month,date
0,2098,2098,13198,14107,1755,1850,-12.0,175.0,1044,0,...,0,0,0,0,0,0,0,2019,3,25
1,2238,2238,13198,14107,2000,2055,-19.0,175.0,1044,0,...,0,0,1,0,0,0,0,2019,3,25
2,2451,2451,13198,14107,540,635,-10.0,175.0,1044,0,...,0,0,2,0,0,0,0,2019,3,25
3,2213,2213,13198,14492,1550,1905,94.0,135.0,904,0,...,0,0,3,0,0,1,1,2019,3,25
4,2096,2096,13198,14635,1045,1430,152.0,165.0,1155,0,...,0,0,4,0,0,2,2,2019,3,25


In [18]:
set(flights_final.columns) ^ set(flights_test.columns)

{'arr_delay'}

In [12]:
X = flights_final.drop('arr_delay', axis = 1)
y = flights_final['arr_delay']

### sklearn

In [13]:
reg = LinearRegression().fit(X, y)

In [14]:
reg.score(X, y)

0.01238503161987592

In [15]:
reg.coef_

array([-2.39532148e-03,  2.80051692e-03, -2.73754387e-04, -1.07119122e-04,
        6.70934775e-03,  3.60701911e-03, -2.17027134e-02,  1.98669931e-03,
        6.96435790e-01, -7.05113789e-01,  6.96435790e-01,  8.70016795e-02,
        3.46353461e-04,  3.48285237e-02, -3.69090916e-02,  3.54394663e-02,
       -4.42136118e-02,  4.95042385e-01, -9.46044984e-02, -1.77355894e-02])

### Statsmodel

In [16]:
res = sm.OLS(y, X).fit()

In [17]:
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:              arr_delay   R-squared (uncentered):                   0.024
Model:                            OLS   Adj. R-squared (uncentered):              0.024
Method:                 Least Squares   F-statistic:                          2.032e+04
Date:                Wed, 26 Aug 2020   Prob (F-statistic):                        0.00
Time:                        14:25:32   Log-Likelihood:                     -8.3110e+07
No. Observations:            15615740   AIC:                                  1.662e+08
Df Residuals:                15615721   BIC:                                  1.662e+08
Df Model:                          19                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------